# Watson's Lemma, for Speed in Maple #

### Robert M. Corless, Michelle Hatzel, and Erik Postma ###

Copyright (c) 2024 Robert M. Corless, Michelle Hatzel, and Erik Postma

[https://doi.org/10.5206/mt.v4i3.21093](https://doi.org/10.5206/mt.v4i3.21093)

See also the Quadrature chapter of Perturbation Methods using backward error, by Corless & Fillion, to be published by SIAM>

Permission is hereby granted, free of charge, to any person obtaining
a copy of this software and associated documentation files (the
"Software"), to deal in the Software without restriction, including
 without limitation the rights to use, copy, modify, merge, publish,
 distribute, sublicense, and/or sell copies of the Software, and to
 permit persons to whom the Software is furnished to do so, subject to
 the following conditions:

 The above copyright notice and this permission notice shall be
 included in all copies or substantial portions of the Software.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
 EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
 MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
 IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY
 CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT,
 TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
 SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.



In [2]:
Watson := proc(f::{operator, procedure}, x::name, {N::posint := Order-1}, $)
  local t, w;
  w := asympt(f(1/t), t, N+1);
  w := eval( convert(w, polynom), t=1/t );
  w := (int(w*exp(-x*t), t = 0 .. infinity) assuming (0 < x));
  w := convert( asympt(expand(w), x, N+1), polynom );
end proc:

In [3]:
f := t -> 1/sqrt(ln(1+t));

t -> 1/sqrt(ln(t+1))

In [26]:
F := Watson(f, x, N=9 ); # guaranteed six figure accuracy for x>=10

            2147483648                     17179869184

$$\sqrt{\pi} \sqrt{\frac{1}{x}}+\frac{\sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{3}{2}}}{8}-\frac{7 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{5}{2}}}{128}+\frac{75 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{7}{2}}}{1024}-\frac{5509 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{9}{2}}}{32768}+\frac{144207 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{11}{2}}}{262144}-\frac{9825299 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{13}{2}}}{4194304}+\frac{412640371 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{15}{2}}}{33554432}-\frac{164900635757 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{17}{2}}}{2147483648}+\frac{9551552651355 \sqrt{\pi} \left(\frac{1}{x}\right)^{\frac{19}{2}}}{17179869184}$$

In [27]:
F := expand(simplify(F)) assuming x>0;

$$\frac{\sqrt{\pi}}{\sqrt{x}}+\frac{\sqrt{\pi}}{8 x^{\frac{3}{2}}}-\frac{7 \sqrt{\pi}}{128 x^{\frac{5}{2}}}+\frac{75 \sqrt{\pi}}{1024 x^{\frac{7}{2}}}-\frac{5509 \sqrt{\pi}}{32768 x^{\frac{9}{2}}}+\frac{144207 \sqrt{\pi}}{262144 x^{\frac{11}{2}}}-\frac{9825299 \sqrt{\pi}}{4194304 x^{\frac{13}{2}}}+\frac{412640371 \sqrt{\pi}}{33554432 x^{\frac{15}{2}}}-\frac{164900635757 \sqrt{\pi}}{2147483648 x^{\frac{17}{2}}}+\frac{9551552651355 \sqrt{\pi}}{17179869184 x^{\frac{19}{2}}}$$

In [44]:
Digits := 18;
CodeGeneration[Julia](codegen[makeproc](evalf(F),x), optimize=true, output=string);

                                 Digits := 18

"function cg1(x)\n    t1 = sqrt(x)\n    t7 = x ^ 2\n    t11 = t7 * x\n    t15 = t7 ^ 2\n    t31 = t15 ^ 2\n    return(0.177245385090551603e1 / t1 + 0.221556731363189504e0 / t1 / x - 0.969310699713954079e-1 / t1 / t7 + 0.129818397283118850e0 / t1 / t11 - 0.297987312763625727e0 / t1 / t15 + 0.975037584219099999e0 / t1 / t15 / x - 0.415203310223772901e1 / t1 / t15 / t7 + 0.217970017974981910e2 / t1 / t15 / t11 - 0.136102906830731073e3 / t1 / t31 + 0.985437438300633891e3 / t1 / t31 / x)\nend\n"

In [28]:
eval(F,x=10.);

$$ 0.567232807559317$$

In [30]:
Frho := simplify(eval(F,x=1/rho^2)) assuming rho>0;

$$\frac{\left(9551552651355 \rho^{19}-1319205086056 \rho^{17}+211271869952 \rho^{15}-40244424704 \rho^{13}+9450749952 \rho^{11}-2888302592 \rho^{9}+1258291200 \rho^{7}-939524096 \rho^{5}+2147483648 \rho^{3}+17179869184 \rho \right) \sqrt{\pi}}{17179869184}$$

In [48]:
FH := convert(evalf(Frho), horner);

$$\left( 1.77245385090551603+\left( 0.221556731363189504+\left(- 0.0969310699713954079+\left( 0.129818397283118850+\left(- 0.297987312763625727+\left( 0.975037584219100000+\left(- 4.15203310223772901+\left( 21.7970017974981910+\left(- 136.102906830731073+ 985.437438300633890 \rho^{2}\right) \rho^{2}\right) \rho^{2}\right) \rho^{2}\right) \rho^{2}\right) \rho^{2}\right) \rho^{2}\right) \rho^{2}\right) \rho^{2}\right) \rho$$

In [49]:
CS := rho=1/sqrt(x), FH;
F1 := codegen[makeproc]( evalf([CS]), x );

        rho ) rho ) rho ) rho ) rho ) rho

proc (x) local rho; rho := 1/x^(1/2); (1.77245385090551603+(.221556731363189504+(-.969310699713954079e-1+(.129818397283118850+(-.297987312763625727+(.975037584219100000+(-4.15203310223772901+(21.7970017974981910+(-136.102906830731073+985.437438300633890*rho^2)*rho^2)*rho^2)*rho^2)*rho^2)*rho^2)*rho^2)*rho^2)*rho^2)*rho end proc

In [50]:
F2 := codegen[optimize]( F1, tryhard );

proc (x) local rho, result, t3; rho := 1/x^(1/2); t3 := rho^2; result := (1.77245385090551603+(.221556731363189504+(-.969310699713954079e-1+(.129818397283118850+(-.297987312763625727+(.975037584219100000+(-4.15203310223772901+(21.7970017974981910+(-136.102906830731073+985.437438300633890*t3)*t3)*t3)*t3)*t3)*t3)*t3)*t3)*t3)*rho end proc

In [51]:
huh := CodeGeneration[Julia]( F2 , output=string);

kilobytes used=42235, alloc=24750, time=5.25

"function F2(x)\n    rho = x ^ (-1//2)\n    t3 = rho ^ 2\n    result = (0.177245385090551603e1 + (0.221556731363189504e0 + (-0.969310699713954079e-1 + (0.129818397283118850e0 + (-0.297987312763625727e0 + (0.975037584219100000e0 + (-0.415203310223772901e1 + (0.217970017974981910e2 + (-0.136102906830731073e3 + 0.985437438300633890e3 * t3) * t3) * t3) * t3) * t3) * t3) * t3) * t3) * t3) * rho\n    return(result)\nend\n"

In [39]:
lprint(huh);

"function F2(x)\n    rho = x ^ (-1//2)\n    t1 = rho ^ 2\n    result = (17179869184 + (2147483648 + (-939524096 + (1258291200 + (-2888302592 + (9450749952 + (-40244424704 + (211271869952 + (-1319205086056 + 9551552651355 * t1) * t1) * t1) * t1) * t1) * t1) * t1) * t1) * t1) * rho * sqrt(pi) / 17179869184\n    return(result)\nend\n"